# Pacoes e classes

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from funcoes.buscar_dados import Temperature, Load
from funcoes.equacoes import  tabela_equacoes
from funcoes.graficos import plotar_graficos, grid_graficos, grid_graficos_estacoes
from funcoes.estacao_ano import estacao_do_ano
from funcoes.funcoes_gerais import preparar_df_long_com_ponderada, trocar_temp
from funcoes.funcoes_teste import treinamento_media_simples, treinamento_estado_total_pond, treinamento_por_weather_zone_e_hora_cv, modelo_estado_temp_media
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import gridstatus as gs
from gridstatus import Ercot
import datetime as dt
import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# credencias
api_key = '1a69aa1efefb4b73a592faf0e2fbe241'
start_date = "2020-01-01"
end_date = "2025-01-01"

In [ ]:
# Temperatura
# temp = Temperature(api_key)
# df_temp = temp.get_df_temp(start_date, end_date)
# df_temp.to_csv('df_temp2.csv')

# # Carga
# load = Load(api_key)
# df_load = load.get_df_load(start_date, end_date)
# df_load.to_csv('df_load2.csv')


# Lendo arquivos baixados pela api
filtrar de 2023/09 até 2024/09 todas as horas da carga

In [3]:
# df_t = pd.read_csv('C:/Users/joao.pinheiro/OneDrive - Serena/Área de Trabalho/df_temp.csv')
# df_l = pd.read_csv('C:/Users/joao.pinheiro/OneDrive - Serena/Área de Trabalho/df_load.csv')

df_t = pd.read_csv('C:/Users/Computadores Gamer/OneDrive/Área de Trabalho/dados_tcc/df_temp.csv')
df_l = pd.read_csv('C:/Users/Computadores Gamer/OneDrive/Área de Trabalho/dados_tcc/df_load.csv')

In [4]:
# dados da populacao do texas
pop_texas = pd.read_csv('C:/Users/Computadores Gamer/OneDrive/Área de Trabalho/serena-tcc/dados/pop_texas.csv')
pop_texas = pop_texas.drop_duplicates(keep = 'first', subset=['COUNTY'])


In [5]:
# Tratamentos iniciais
for df in [df_l, df_t]:
    df["interval_start_utc"] = pd.to_datetime(df["interval_start_utc"]).dt.tz_localize(None)
    df["interval_end_utc"] = pd.to_datetime(df["interval_end_utc"], utc=True).dt.tz_localize(None)
    df["publish_time_utc"] = pd.to_datetime(df["publish_time_utc"], utc=True).dt.tz_localize(None)
    
df_t = df_t.drop(columns=["Unnamed: 0"], errors="ignore")
df_l = df_l.drop(columns=["Unnamed: 0", "publish_time_utc_load",'publish_time_utc_tf'], errors="ignore")


df_t = df_t[['interval_start_utc', 'interval_end_utc', 'publish_time_utc', 'coast',
        'east', 'far_west', 'north', 'north_central', 'south_central',
        'southern', 'west', 'hour', 'weekday', 'avg_temp_fahrenheit']]


In [6]:
df = pd.merge(
    df_l,
    df_t,
    on=["interval_start_utc", "interval_end_utc"],
    how="inner",
    suffixes=['_carga','_tf']
)

df = trocar_temp(df)
df['publish_time_utc_carga'] = pd.to_datetime(df['publish_time_utc_carga'])


hora = dt.time(10, 30) 
df = df[(df['publish_time_utc_carga'].dt.time == hora)]


# Criacao da coluna de temperatura ponderada pela populacao

In [49]:
# Dicionário de população por região
populacao_regioes = {
    'coast': 1000000,
    'east': 800000,
    'far_west': 200000,
    'north': 900000,
    'north_central': 1100000,
    'south_central': 950000,
    'southern': 700000,
    'west': 500000
}

# Lista das colunas de temperatura
colunas_temp = [f'{regiao}_tc' for regiao in populacao_regioes]

# Calcula o numerador (temperatura * população por região)
numerador = sum(df[col] * populacao_regioes[col.rsplit('_', 1)[0]] for col in colunas_temp)

# Soma total das populações
denominador = sum(populacao_regioes.values())

# Cria a nova coluna de temperatura ponderada
df['temp_ponderada_pop'] = numerador / denominador

In [54]:
df.columns

Index(['interval_start_utc', 'interval_end_utc', 'publish_time_utc_carga',
       'coast_carga', 'east_carga', 'far_west_carga', 'north_carga',
       'north_central_carga', 'south_central_carga', 'southern_carga',
       'west_carga', 'system_total', 'sum_load', 'publish_time_utc_tf',
       'coast_tf', 'east_tf', 'far_west_tf', 'north_tf', 'north_central_tf',
       'south_central_tf', 'southern_tf', 'west_tf', 'hour', 'weekday',
       'avg_temp_fahrenheit', 'coast_tc', 'east_tc', 'far_west_tc', 'north_tc',
       'north_central_tc', 'south_central_tc', 'southern_tc', 'west_tc',
       'avg_temp_celsius', 'temp_ponderada_pop'],
      dtype='object')

In [68]:
def treinamento(df, column_x, column_y, k=5):
    resultados = []

    horas = sorted(df['hour'].unique())

    for hora in horas:
        # Filtra os dados para a hora atual
        dados = df[df['hour'] == hora]
        dados = dados.dropna(subset=[column_x, column_y])

        X = dados[[column_x]].values
        y = dados[column_y].values

        # Divide em treino, teste e validação
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
        X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

        # Regressão quadrática: x², x, constante
        X_train_poly = np.hstack((X_train**2, X_train, np.ones_like(X_train)))
        coef, *_ = np.linalg.lstsq(X_train_poly, y_train, rcond=None)

        # Previsões
        y_pred_train = X_train_poly @ coef
        X_test_poly = np.hstack((X_test**2, X_test, np.ones_like(X_test)))
        y_pred_test = X_test_poly @ coef
        X_val_poly = np.hstack((X_val**2, X_val, np.ones_like(X_val)))
        y_pred_val = X_val_poly @ coef

        # Métricas de erro
        rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
        rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
        rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

        r2_train = r2_score(y_train, y_pred_train)
        r2_test = r2_score(y_test, y_pred_test)
        r2_val = r2_score(y_val, y_pred_val)

        # Validação cruzada
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        rmse_cv_scores = []

        for train_idx, test_idx in kf.split(X_val):
            X_cv_train, X_cv_test = X_val[train_idx], X_val[test_idx]
            y_cv_train, y_cv_test = y_val[train_idx], y_val[test_idx]

            X_cv_train_poly = np.hstack((X_cv_train**2, X_cv_train, np.ones_like(X_cv_train)))
            X_cv_test_poly = np.hstack((X_cv_test**2, X_cv_test, np.ones_like(X_cv_test)))

            coef_cv, *_ = np.linalg.lstsq(X_cv_train_poly, y_cv_train, rcond=None)
            y_cv_pred = X_cv_test_poly @ coef_cv
            rmse_cv_scores.append(mean_squared_error(y_cv_test, y_cv_pred, squared=False))

        # Cálculo do erro relativo da validação cruzada
        rmse_cv_mean = np.mean(rmse_cv_scores)
        rmse_cv_std = np.std(rmse_cv_scores)

        resultados.append({
            'hora': hora,
            'equacao': f"y = {coef[0]:.2f}x² {coef[1]:+.2f}x {coef[2]:+.2f}",
            'RMSE Treino': rmse_train,
            'R² Treino': r2_train,
            'RMSE Teste': rmse_test,
            'R² Teste': r2_test,
            'RMSE Validação': rmse_val,
            'R² Validação': r2_val,
            # 'CV RMSE Médio': rmse_cv_mean,
            # 'CV RMSE DP': rmse_cv_std
        })

    return pd.DataFrame(resultados)

tabela1 = treinamento(df, 'avg_temp_celsius', 'sum_load')
tabela1
tabela2 = treinamento(df, 'temp_ponderada_pop', 'sum_load')
tabela2




,hora,equacao,RMSE Treino,R² Treino,RMSE Teste,R² Teste,RMSE Validação,R² Validação
0,0,y = 93.38x² -2526.62x +55949.72,2216.441053,0.925512,1865.425720,0.944885,2263.012506,0.912739
1,1,y = -4.09x² +723.41x +33344.39,5875.154668,0.370598,5072.645419,0.506422,5837.987592,0.301505
2,2,y = 92.29x² -2511.94x +53887.20,2037.779981,0.909229,1929.763840,0.914888,2031.727365,0.904018
3,3,y = 91.47x² -2474.20x +53257.70,1973.879240,0.901041,1603.234874,0.930224,2180.739795,0.865318
4,4,y = 91.30x² -2474.72x +53643.12,1973.316262,0.887092,1584.931906,0.920234,2249.955179,0.837097
5,5,y = 92.36x² -2546.50x +55836.78,2150.013834,0.849090,1807.020632,0.878095,2458.851350,0.782238
6,6,y = 89.81x² -2540.39x +58820.95,2732.179469,0.730202,2429.314998,0.746271,3003.419613,0.648687
7,7,y = 87.93x² -2517.38x +60320.90,2970.975255,0.666645,2686.209277,0.679003,3236.112598,0.583207
8,8,y = 87.33x² -2516.73x +61152.85,2649.976121,0.763899,2614.061241,0.772475,2425.616524,0.760015
9,9,y = 92.19x² -2814.47x +65491.68,2496.166520,0.851097,2328.212764,0.870030,2359.963767,0.844478


In [73]:
def treinamento_todas_regioes(df, k=5):
    resultados = []

    # Definindo as colunas de carga e temperatura para cada região
    regioes = {
        'coast': ('coast_carga', 'coast_tc'),
        'east': ('east_carga', 'east_tc'),
        'far_west': ('far_west_carga', 'far_west_tc'),
        'north': ('north_carga', 'north_tc'),
        'north_central': ('north_central_carga', 'north_central_tc'),
        'south_central': ('south_central_carga', 'south_central_tc'),
        'southern': ('southern_carga', 'southern_tc'),
        'west': ('west_carga', 'west_tc')
    }

    horas = sorted(df['hour'].unique())

    # Itera sobre as horas
    for hora in horas:
        # Inicializa a lista de resultados para cada hora
        resultado_hora = {'hora': hora}

        # Itera sobre as regiões
        for regiao, (col_carga, col_temp) in regioes.items():
            # Filtra os dados para a hora e a região atual
            dados = df[df['hour'] == hora]
            dados = dados.dropna(subset=[col_carga, col_temp])  # Filtrando os dados

            X = dados[[col_temp]].values  # Temperatura da região
            y = dados[col_carga].values  # Carga da região

            # Divide em treino, teste e validação
            X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
            X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

            # Regressão quadrática: x², x, constante
            X_train_poly = np.hstack((X_train**2, X_train, np.ones_like(X_train)))
            coef, *_ = np.linalg.lstsq(X_train_poly, y_train, rcond=None)

            # Previsões
            y_pred_train = X_train_poly @ coef
            X_test_poly = np.hstack((X_test**2, X_test, np.ones_like(X_test)))
            y_pred_test = X_test_poly @ coef
            X_val_poly = np.hstack((X_val**2, X_val, np.ones_like(X_val)))
            y_pred_val = X_val_poly @ coef

            # Métricas de erro
            rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
            rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
            rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)

            r2_train = r2_score(y_train, y_pred_train)
            r2_test = r2_score(y_test, y_pred_test)
            r2_val = r2_score(y_val, y_pred_val)
            
            # Adiciona a equação para a região
            resultado_hora[f'Equação {regiao}'] = f"y = {coef[0]:.2f}x² {coef[1]:+.2f}x {coef[2]:+.2f}"

            # Adiciona as métricas para a região
            resultado_hora[f'RMSE {regiao} Treino'] = rmse_train
            resultado_hora[f'R² {regiao} Treino'] = r2_train
            resultado_hora[f'RMSE {regiao} Teste'] = rmse_test
            resultado_hora[f'R² {regiao} Teste'] = r2_test
            resultado_hora[f'RMSE {regiao} Validação'] = rmse_val
            resultado_hora[f'R² {regiao} Validação'] = r2_val



        # Adiciona os resultados para a hora
        resultados.append(resultado_hora)

    return pd.DataFrame(resultados)


tabela3 = treinamento_todas_regioes(df)
tabela3

,hora,Equação coast,RMSE coast Treino,R² coast Treino,RMSE coast Teste,R² coast Teste,RMSE coast Validação,R² coast Validação,Equação east,RMSE east Treino,...,R² southern Teste,RMSE southern Validação,R² southern Validação,Equação west,RMSE west Treino,R² west Treino,RMSE west Teste,R² west Teste,RMSE west Validação,R² west Validação
0,0,y = 26.22x² -751.19x +15227.30,790.679255,0.873037,659.070029,0.910780,662.042813,0.901513,y = 3.78x² -109.71x +2119.17,147.100806,...,0.901601,207.833911,0.895551,y = 1.98x² -54.35x +1467.12,110.899211,0.744019,96.695698,0.784152,111.556781,0.724798
1,1,y = -4.33x² +375.63x +5991.43,1490.702950,0.470530,1254.710646,0.615542,1444.961746,0.441427,y = 0.37x² -5.92x +1489.70,254.534573,...,0.630459,464.109162,0.413591,y = 0.59x² -11.67x +1228.63,176.079680,0.252497,150.404917,0.431597,178.636078,0.240642
2,2,y = 23.92x² -678.52x +14208.36,711.575953,0.859468,720.051023,0.848348,650.588624,0.868700,y = 3.66x² -108.01x +2039.01,143.314098,...,0.878059,211.561943,0.849533,y = 2.04x² -56.93x +1446.39,115.369885,0.636177,111.106112,0.590819,117.503683,0.624947
3,3,y = 23.39x² -663.30x +14044.64,733.045474,0.834488,577.482539,0.890589,631.806290,0.857925,y = 3.56x² -104.50x +2000.31,137.577499,...,0.867114,194.710281,0.866711,y = 2.05x² -57.01x +1430.91,110.335091,0.619526,108.126005,0.622319,113.408793,0.618128
4,4,y = 23.03x² -654.15x +14114.06,726.539319,0.821179,572.566255,0.877925,653.595792,0.829173,y = 3.56x² -105.91x +2037.25,135.232355,...,0.863606,185.749160,0.856649,y = 2.04x² -57.04x +1442.78,108.271200,0.578662,107.670625,0.590181,114.201441,0.577306
5,5,y = 23.94x² -697.63x +14935.84,769.975335,0.784752,638.134907,0.826518,705.375433,0.782583,y = 3.60x² -109.16x +2139.50,137.855741,...,0.830312,174.352604,0.848487,y = 1.97x² -56.35x +1486.66,110.142771,0.517418,110.095520,0.557141,117.290719,0.518873
6,6,y = 23.50x² -700.41x +15644.63,867.450694,0.700003,745.356096,0.714158,826.902897,0.669870,y = 3.41x² -106.99x +2271.72,159.772034,...,0.680902,212.863789,0.734136,y = 1.76x² -52.63x +1553.36,123.235906,0.406167,126.675762,0.491084,130.299802,0.444063
7,7,y = 24.84x² -754.46x +16395.92,893.683074,0.677787,785.925734,0.672375,861.129364,0.640983,y = 3.32x² -105.09x +2326.39,162.678675,...,0.614025,230.091343,0.672977,y = 1.70x² -51.90x +1606.93,131.750242,0.385464,143.195579,0.448743,141.479772,0.408911
8,8,y = 24.83x² -760.56x +16703.37,879.270718,0.744753,764.473635,0.794558,716.133368,0.778786,y = 3.31x² -104.47x +2348.06,148.936391,...,0.809453,181.725927,0.819807,y = 1.59x² -46.28x +1588.87,126.448052,0.345874,132.294701,0.444218,119.851022,0.376806
9,9,y = 25.48x² -812.46x +17684.70,923.083764,0.799051,790.677391,0.845928,758.931912,0.833551,y = 3.72x² -122.66x +2562.75,134.122401,...,0.885855,194.200776,0.867709,y = 1.69x² -49.35x +1641.44,122.778309,0.460053,115.511747,0.549230,116.528164,0.492008
